In [2]:
import os
import warnings
from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
_ = load_dotenv(find_dotenv())

In [3]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from operator import itemgetter
from langchain.schema.runnable import RunnableBranch

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
chat_model = ChatGroq(
    model="llama3-70b-8192"
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# template  = ChatPromptTemplate.from_template("tell me a curious fact about this politician {politician}")

# chain = template | chat_model | StrOutputParser()

# response = chain.invoke({"politician":"Akhnouch"})

# for s in chain.stream({"politician": "akhnouch"}):
#     print(s, end="", flush=True)

c:\Users\lenovo\anaconda3\envs\genAI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# chain.batch([{"politician": "Lenin"}, {"politician": "Stalin"}])

['Here\'s a curious fact about Vladimir Lenin:\n\n**Lenin was a passionate beekeeper!**\n\nBefore he became the leader of the Soviet Union, Lenin spent many summers at his family\'s country estate in Simbirsk, Russia, where he developed a fascination with beekeeping. In fact, he was so enthusiastic about it that he even wrote an article about beekeeping in 1893, titled "On the Importance of Beekeeping in Russia."\n\nLenin saw beekeeping as a way to promote agricultural development and improve the lives of peasants. He believed that beekeeping could provide a supplementary income for rural communities and help to increase food production.\n\nLenin\'s love for beekeeping was so strong that even after the Russian Revolution, he continued to keep bees at his country home in Gorki, near Moscow. In fact, his beekeeping hobby was often mentioned in his letters to friends and family.\n\nWho would have thought that the leader of the Soviet Union was also a bee enthusiast?',
 "Here's a curious f

### Runnables

In [4]:
chain = RunnablePassthrough()
chain.invoke("ayoub")

'ayoub'

In [6]:
# To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
# Let's define a very simple function to create Russian lastnames:

def russian_lastname(name: str) -> str:
    return f"{name}ovich"


chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("ayoub")

'ayoubovich'

In [7]:
# We will use RunnableParallel() for running tasks in parallel.
# This is probably the most important and most useful Runnable from LangChain.


chain = RunnableParallel(
    {
        "operation_a": RunnableLambda(russian_lastname),
        "operation_b": RunnablePassthrough()
    }
)

chain.invoke("ayoub")

{'operation_a': 'ayoubovich', 'operation_b': 'ayoub'}

In [8]:
# Instead of using RunnableLambda, now we are going to use a lambda function and we will invoke the chain with two inputs:

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": lambda x: x["name"]+"ovich"
    }
)

chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

{'operation_a': {'name1': 'Jordam', 'name': 'Abram'},
 'soccer_player': 'Abramovich'}

In [14]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
    }
) | prompt | chat_model | output_parser

response = chain.invoke({
    "name1": "Jordam",
    "name": "Ibrahim"
})
print(response)

Here's one:

Did you know that Zlatan Ibrahimović, the Swedish professional footballer, has a black belt in Taekwondo? In fact, he earned his black belt at the age of 17, before he even became a professional footballer! He has often credited his martial arts background with helping him develop his balance, agility, and overall athleticism on the pitch. Who knew that one of the most feared strikers in football history was also a skilled martial artist?


In [15]:
vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 10,000 Alumni from all continents and top companies"], embedding=embeddings
)

In [16]:
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retriever_chain = (
    {"context":retriever,"question":RunnablePassthrough()}
    | prompt
    | chat_model
    |StrOutputParser()
)

retriever_chain.invoke("who are the Alumni of AI Accelera?")

'Based on the provided context, the Alumni of AI Accelera are individuals from all continents and top companies.'

In [22]:

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 5,000 Enterprise Alumni."], embedding=embeddings
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | chat_model
    | StrOutputParser()
)
chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

"Arrr, AI Accelera be trainin' more than 5,000 scurvy Enterprise Alumni, matey!"

In [24]:
from langchain.prompts import PromptTemplate

rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""

rock_prompt = PromptTemplate.from_template(rock_template)

politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""

politics_prompt = PromptTemplate.from_template(politics_template)

history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""

history_prompt = PromptTemplate.from_template(history_template)

sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

sports_prompt = PromptTemplate.from_template(sports_template)

In [25]:

general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)

prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "rock", rock_prompt),
  (lambda x: x["topic"] == "politics", politics_prompt),
  (lambda x: x["topic"] == "history", history_prompt),
  (lambda x: x["topic"] == "sports", sports_prompt),
  general_prompt
)

In [26]:
from typing import Literal

from pydantic import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain_core.utils.function_calling import convert_to_openai_function

class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    
    topic: Literal["rock", "politics", "history", "sports", "general"]
    "The topic of the user question. One of 'rock', 'politics', 'history', 'sports' or 'general'."

classifier_function = convert_to_openai_function(TopicClassifier)

llm = chat_model.bind(functions=[classifier_function], function_call={"name": "TopicClassifier"}) 

parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = llm | parser

In [27]:
a = classifier_function
a

{'name': 'TopicClassifier',
 'description': 'Classify the topic of the user question',
 'parameters': {'properties': {'topic': {'enum': ['rock',
     'politics',
     'history',
     'sports',
     'general'],
    'type': 'string'}},
  'required': ['topic'],
  'type': 'object'}}

In [28]:
question = "they have killed the villain"
response = classifier_chain.invoke(question)
response

'general'

In [45]:
final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch 
    | chat_model
    | StrOutputParser()
)

In [46]:
final_chain.invoke(
{"input":"Who was Napoleon Bonaparte?"}
)

"What a fantastic question! Napoleon Bonaparte, one of the most fascinating and influential figures in modern history. Born on August 15, 1769, in Ajaccio, Corsica, Napoleon was a French military and political leader who rose to prominence during the French Revolution and went on to dominate European politics for over a decade.\n\nInitially, Napoleon was a brilliant and ambitious military tactician who rose through the ranks of the French army during the Revolution. He gained fame and recognition for his military victories, particularly in Italy and Egypt, which earned him the respect and admiration of the French people.\n\nHowever, Napoleon's ambition soon turned to politics, and in 1799, he staged a coup d'état, overthrowing the Directory, the governing body of France, and establishing the Consulate, with himself as the First Consul. This marked the beginning of his dictatorship, which would last until his final defeat in 1815.\n\nAs Emperor of France, Napoleon implemented a range of

### Explain asign() and bind()

In [47]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

chain = prompt | chat_model | StrOutputParser()

print(chain.invoke({"soccer_player": "Ronaldo"}))

Here's one:

**Ronaldo's iconic celebration, where he jumps and spins in the air, was inspired by a video game!**

In an interview, Ronaldo revealed that he got the idea for his signature celebration from playing the video game " FIFA" as a kid. He would play as himself in the game and celebrate his goals with a fancy flip, and eventually, it became a part of his real-life celebrations on the pitch!

Isn't that cool?


In [48]:
chain = prompt | chat_model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

"Here's one:\n\n**Cristiano "

In [49]:
functions = [
    {
      "name": "soccerfacts",
      "description": "Curious facts about a soccer player",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question for the curious facts about a soccer player"
          },
          "answer": {
            "type": "string",
            "description": "The answer to the question"
          }
        },
        "required": ["question", "answer"]
      }
    }
  ]

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | chat_model.bind(function_call={"name": "soccerfacts"}, functions= functions)
    | JsonOutputFunctionsParser()
)


chain.invoke({"soccer_player": "Mbappe"})

{'question': 'What is a curious fact about Kylian Mbappe?', 'answer': ''}

In [4]:
def make_uppercase(arg):
    return arg["original_input"].upper()

chain = RunnableParallel({"original_input": RunnablePassthrough()}).assign(uppercase=RunnableLambda(make_uppercase))
chain.invoke("whatever")

{'original_input': 'whatever', 'uppercase': 'WHATEVER'}

In [5]:
chain = (
    RunnablePassthrough() | make_uppercase
    )
chain.invoke({"original_input":"ayoub"})

'AYOUB'

### Chain inside chain

In [24]:
prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

chain1 = prompt1 | chat_model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | chat_model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"Vous pensez à la France, et la France est située sur le continent européen.\n\n(Translation: You're thinking of France, and France is located on the European continent.)"

### Fallback for chains

In [7]:
model = ChatGroq(model="llama3-70b-8192")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_spliter.split_documents(docs)

vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

retriever = vectordb.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context":retriever | RunnableLambda(format_docs) ,"question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
response = rag_chain.invoke("What is Task Decomposition?")

print(response)

Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. It involves decomposing hard tasks into simpler steps, making it easier for an agent or model to plan and execute the task. This can be done through various methods, including Chain of Thought, Tree of Thoughts, and simple prompting or task-specific instructions.


### Memory

In [15]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15756\1046716321.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [16]:
conversation({"question": "My name is Julio and I have moved 33 times."})

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15756\533826746.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation({"question": "My name is Julio and I have moved 33 times."})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: My name is Julio and I have moved 33 times.

> Finished chain.


{'question': 'My name is Julio and I have moved 33 times.',
 'chat_history': [HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Wow, Julio! 33 times is a lot of moving! I can only imagine how many different places you've lived in and the experiences you've had. That's incredible!\n\nWhat's been the most challenging part of moving so many times for you? Was it adjusting to new schools, making new friends, or something else entirely?\n\nAnd if you don't mind me asking, what's been the reason behind all these moves? Was it for work, family, or something else?", additional_kwargs={}, response_metadata={})],
 'text': "Wow, Julio! 33 times is a lot of moving! I can only imagine how many different places you've lived in and the experiences you've had. That's incredible!\n\nWhat's been the most challenging part of moving so many times for you? Was it adjusting to new schools, making new friends, or something el

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | chat_model

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [20]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is the ratio of the adjacent side to the hypotenuse in a right-angled triangle.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 37, 'total_tokens': 59, 'completion_time': 0.069623428, 'prompt_time': 0.001151258, 'queue_time': 0.014330572, 'total_time': 0.070774686}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-2f560d2e-5705-4615-8c51-7fe562e14956-0', usage_metadata={'input_tokens': 37, 'output_tokens': 22, 'total_tokens': 59})

In [21]:
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine = adjacent side / hypotenuse.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 70, 'total_tokens': 81, 'completion_time': 0.034728488, 'prompt_time': 0.003179872, 'queue_time': 0.012308707, 'total_time': 0.03790836}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-df29bb32-8c93-48c6-90e9-818a35dcee7f-0', usage_metadata={'input_tokens': 70, 'output_tokens': 11, 'total_tokens': 81})

In [22]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content="I'm here to help with math problems! What's the question or problem you need assistance with?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 34, 'total_tokens': 55, 'completion_time': 0.06, 'prompt_time': 0.004369841, 'queue_time': 0.009173239, 'total_time': 0.064369841}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd6e63bc-244e-4824-b936-5d40d1ff0da7-0', usage_metadata={'input_tokens': 34, 'output_tokens': 21, 'total_tokens': 55})